# [모듈 2.1] CodeBuild 프로젝트 생성

# 1. 환경 설정 

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('./src')

## 1.0 기존의 변수 불러오기

In [ ]:
%store -r code_pipeline_serving_config_json_path
%store -r sm_pipeline_serving_config_json_path

from common_utils import load_json

code_pipeline_serving_dict = load_json(code_pipeline_serving_config_json_path)
sm_pipeline_serving_dict = load_json(sm_pipeline_serving_config_json_path)

import json
print("Code Pipeline Series Params: ")
print (json.dumps(code_pipeline_serving_dict, indent=2))
print("SageMaker Pipeline Series Params: ")
print (json.dumps(sm_pipeline_serving_dict, indent=2))



## 1.1. 파라미터 설정

In [ ]:
import sagemaker
import boto3

region = sagemaker.Session().boto_region_name

account_id = boto3.client('sts').get_caller_identity().get('Account')

build_service_role = code_pipeline_serving_dict["code_build_service_arn"]
region = code_pipeline_serving_dict["region"]
account_id = code_pipeline_serving_dict["account_id"]
bucket = code_pipeline_serving_dict["bucket"]
model_package_group_name = code_pipeline_serving_dict["model_package_group_name"]
endpoint_name = code_pipeline_serving_dict["endpoint_name"]
repository_name = code_pipeline_serving_dict["serving_code_repo_name"]
project_prefix = code_pipeline_serving_dict["project_prefix"]
code_build_project_name = code_pipeline_serving_dict["code_build_project_name"]


sm_pipeline_name = sm_pipeline_serving_dict["sm_pipeline_name"]


print("region: ", region)
print("account_id: ", account_id)
print("project_prefix: ", project_prefix)
print("code_build_project_name: ", code_build_project_name)
print("sm_pipeline_name: ", sm_pipeline_name)
print("model_package_group_name: ", model_package_group_name)
print("endpoint_name: ", endpoint_name)

# 2. 빌드 프로젝트  JSON 정의
CodeBuild serviceRole 과 SAGEMAKER_PIPELINE_ROLE_ARN 을 동일한 Role 로 사용함. 이 부분은 실제 적용시에 분리 하셔서 사용을 하셔도 됩니다.

```
# SAGEMAKER_PIPELINE_ROLE_ARN
"name": "SAGEMAKER_PIPELINE_ROLE_ARN",
"value": build_service_role,
```

```
# CodeBuild serviceRole
  "serviceRole": build_service_role,  
```

In [ ]:
build_dic = {
  "name": code_build_project_name,
  "description": "Hello Build Project",
  "source": {
    "type": "CODEPIPELINE",
    "buildspec": "codebuild-buildspec.yml",
  },
  "artifacts": {
    "type": "CODEPIPELINE",
    "name": code_build_project_name,
  },
  "cache": {
    "type": "NO_CACHE",
  },
  "environment": {
            "type": "LINUX_CONTAINER",
            "image": "aws/codebuild/amazonlinux2-x86_64-standard:3.0",
            "computeType": "BUILD_GENERAL1_SMALL",
            "environmentVariables": [
                {
                    "name": "IMAGE_REPO_NAME",
                    "value": "NCF-Serving-Pipelinel",
                    "type": "PLAINTEXT"
                },
                {
                    "name": "IMAGE_TAG",
                    "value": "latest",
                    "type": "PLAINTEXT"
                },
                {
                    "name": "AWS_ACCOUNT_ID",
                    "value": account_id,
                    "type": "PLAINTEXT"
                },
                {
                    "name": "SAGEMAKER_PIPELINE_ROLE_ARN",
                    "value": build_service_role,
                    "type": "PLAINTEXT"
                },        
                {
                    "name": "SAGEMAKER_PIPELINE_NAME",
                    "value": sm_pipeline_name,
                    "type": "PLAINTEXT"
                },                        
                {
                    "name": "AWS_DEFAULT_REGION",
                    "value": region,
                    "type": "PLAINTEXT"
                },
                {
                    "name": "TEMPLATE_BUCKET",
                    "value": bucket,
                    "type": "PLAINTEXT"
                },
                {
                    "name": "code_repository_name",
                    "value": repository_name,
                    "type": "PLAINTEXT"
                },                
                {
                    "name": "project_prefix",
                    "value": project_prefix,
                    "type": "PLAINTEXT"
                },
                {
                    "name": "model_package_group_name",
                    "value": model_package_group_name,
                    "type": "PLAINTEXT"
                },                                
                {
                    "name": "endpoint_name",
                    "value": endpoint_name,
                    "type": "PLAINTEXT"
                },                
                {
                    "name": "TEMPLATE_PREFIX",
                    "value": "codebuild",
                    "type": "PLAINTEXT"
                }
                
            ],
            "privilegedMode": False,
            "imagePullCredentialsType": "CODEBUILD"
        },
  "serviceRole": build_service_role,
  "timeoutInMinutes": 60,
  "queuedTimeoutInMinutes": 480,
  "badgeEnabled": False,
  "logsConfig": {
    "cloudWatchLogs": {
      "status": "ENABLED",
    },
    "s3Logs": {
      "status": "DISABLED",
      "encryptionDisabled": False
    }
  },
}

## 2.1. Dic 포맷을 JSON 으로 변경

In [ ]:
import json

build_json = json.dumps(build_dic)
json.loads(build_json)

## 2.2. JSON 파일 저장

In [ ]:
json_file_path = 'src/hello_build.json'
with open(json_file_path, "w") as outfile:
    outfile.write(build_json)

In [ ]:
with open(json_file_path) as json_file:
    json_data = json.load(json_file)
    
json_data    

# 3. 빌드 프로젝트 생성

In [ ]:
%%sh -s {json_file_path}
json_file_path=$1
echo $json_file_path
aws codebuild create-project --cli-input-json file://$json_file_path

# 4. Code Build 프로젝트 생성 확인

![ncf_serving_code_build.png](img/ncf_serving_code_build.png)